<a href="https://colab.research.google.com/github/vivek1may/DeepLearning/blob/master/Mnist_TF2_0_vs_PT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST TF v2.0 vs PyTorch

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  print(Exception)
  pass

import tensorflow as tf

In [0]:
#Load data
mnist = tf.keras.datasets.mnist
(x_train,y_train), (x_test,y_test) = mnist.load_data() 


In [0]:
#normalize the data
x_train , x_test = x_train/255, x_test/255

In [0]:
#Build a sequential model 

model = tf.keras.models.Sequential([
                                    tf.keras.layers.Flatten(input_shape=(28,28)),
                                    tf.keras.layers.Dense(128,activation='relu'),
                                    tf.keras.layers.Dropout(.2),
                                    tf.keras.layers.Dense(10,activation='softmax')     
])

In [0]:
#Compile the model with optimizer and loss function 

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
#Train the model 
model.fit(x_train,y_train,epochs=5)

In [0]:
model.evaluate(x_test,y_test)

In [0]:
#Pytorch 

In [0]:
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset

In [0]:
#custom DataSet

class testData(Dataset):
  def __init__(self,x,y):
    self.x = x
    self.y = y

  def __len__(self):
    return len(self.x)

  def __getitem__(self,idx):
    return self.x[idx], self.y[idx]


class trainData(Dataset):
  def __init__(self,x,y):
    self.x = x
    self.y = y

  def __len__(self):
    return len(self.x)

  def __getitem__(self,idx):
    return self.x[idx], self.y[idx]


In [0]:
#Initialize the Dataset1
x_train1, y_train1, x_test1, y_test1 = np.float32(x_train), np.float32(y_train), np.float32(x_test), np.float32(y_test)
train_data = trainData(x_train1,y_train)
test_data = testData(x_test1,y_test)

In [0]:
# Initialize the Dataloader
train_loader =  DataLoader(train_data, batch_size=32)
test_loader = DataLoader(test_data, batch_size=32)

In [0]:
#Define the model equivalent to Tensorflow model which created above 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [0]:
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.fc1 = nn.Linear(28*28,128)
    self.fc2 = nn.Linear(128,10)
    self.dropout = nn.Dropout(.2)

  def forward(self,x):
    x = x.view(x.shape[0],-1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dropout(x)
    x = self.fc2(x)
    op = F.log_softmax(x,dim=1)

    return op

In [0]:
device = torch.device("cuda")

In [0]:
model = Net().to(device)

In [0]:
learning_rage = .1
optimizer = optim.Adam(model.parameters())
#Using negative log likelihood 
lossFun = F.nll_loss

In [0]:
def train(model, device, train_loader, optimizer,lossFun, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.long().to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = lossFun(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [0]:
def test(model,device,test_loader,lossFun):
  model.eval()
  test_loss =0
  correct = 0

  #disable the grad
  with torch.no_grad():
    for data,target in test_loader:
      data, target = data.to(device), target.long().to(device)
      output = model(data)
      test_loss += lossFun(output,target,reduction='sum').item()
      pred = output.argmax(dim=1, keepdim =True)
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)

  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)))

In [0]:
epochs = 5
for i in range(epochs):
  train(model,device,train_loader,optimizer,lossFun,i)
  test(model,device,test_loader,lossFun)
